https://console.developers.google.com/apis/api/bigquery.googleapis.com/overview?project=data-339617
https://console.developers.google.com/apis/api/drive.googleapis.com/overview?project=data-339617

1. Create Google Cloud Account
2. Create a service Account
3. create a key json
4. enable apis
5. give access to service account for folders

Data Studio : https://datastudio.google.com/u/0/reporting/9de5f535-abf0-4bc8-9150-6914f4686093/page/p_j9u1xglqrc/edit

Google Sheet: https://docs.google.com/spreadsheets/d/1BMm7WdEtahSsBLV9jAKzlNHyMfU4NwqNrXcSaV5IlFI/edit?usp=sharing

In [1]:
import os, time
import io
from pytz import timezone
localtz = timezone('Europe/Istanbul')
from datetime import datetime, timezone
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from google.cloud import bigquery
import pandas as pd
import numpy as np
from scipy import stats

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def insertintobq_keywordResult(dataset_id,table_name,result):
    KEY_FILE_LOCATION = "key.json"
    
    # establish a BigQuery client
    client = bigquery.Client.from_service_account_json(KEY_FILE_LOCATION)

    # create a job config
    """
    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("Keyword", "STRING"),
            bigquery.SchemaField("LandingPage", "STRING"),
            bigquery.SchemaField("QuestionId", "INTEGER"),
            bigquery.SchemaField("Title", "STRING"),
            bigquery.SchemaField("SearchTryCount", "INTEGER"),
            bigquery.SchemaField("Ip", "STRING"),
            bigquery.SchemaField("Agent", "STRING"),
            bigquery.SchemaField("ChannelId", "INTEGER"),
            bigquery.SchemaField("reportDate", "TIMESTAMP"),
        ]
    )
    """

    job_config = bigquery.LoadJobConfig()

    # Set the destination table
    table_ref = client.dataset(dataset_id).table(table_name)
    #job_config.destination = table_ref
    # job_config.write_disposition = "WRITE_APPEND"
    job_config.write_disposition = "WRITE_TRUNCATE"
    load_job = client.load_table_from_dataframe(result, table_ref, job_config=job_config)
    load_job.result()
    print("tamamlandı")

In [3]:
def customersInsert():
    global customers
    file = "file_customers.csv"

    headers = ['customer_id', 'customer_city', 'channel']
    dtypes = {'customer_id': 'str', 'customer_city': 'str', 'channel': 'str'}
    parse_dates = []

    customers = pd.read_csv(file, dtype=dtypes, parse_dates=parse_dates)
    
    # Detect and exclude outliers in a pandas DataFrame
    customers = customers.drop_duplicates(keep='first')

    dataset_id = "EditedReports"
    table_name = "Customer"
    insertintobq_keywordResult(dataset_id,table_name,customers)
    return

In [4]:
def ordersInsert():
    global orders
    file = "file_orders.csv"

    # headers = ["order_id","customer_id","order_status","order_purchase_timestamp","order_approved_at","order_delivered_carrier_date","order_item_id","product_id","seller_id","price","freight_value","currency","product_category_name","product_name_lenght","product_description_lenght","product_photos_qty","product_weight_g","product_length_cm","product_height_cm","product_width_cm"]
    dtypes = {"order_id":'str',"customer_id":'str',"order_status":'str',"order_item_id":'Int64',"product_id":'str',"seller_id":'str',"price":'float',"freight_value":'float',"currency":'str',"product_category_name":'str',"product_name_lenght":'Int64',"product_description_lenght":'Int64',"product_photos_qty":'Int64',"product_weight_g":'Int64',"product_length_cm":'Int64',"product_height_cm":'Int64',"product_width_cm":'Int64'}
    parse_dates = ["order_approved_at","order_purchase_timestamp","order_delivered_carrier_date"]

    # orders = pd.read_csv(file, sep=',', header=0, names=headers, dtype=dtypes, parse_dates=parse_dates)
    orders = pd.read_csv(file, sep=',', dtype=dtypes, parse_dates=parse_dates)

    # Detect and exclude outliers in a pandas DataFrame  -->productId: 241241093bd6986615756b4d8a01de8f
    # orders["std"]=(np.abs(stats.zscore(orders["price"])))
    # orders.loc[orders['product_id'] == '241241093bd6986615756b4d8a01de8f']
    orders = orders[(np.abs(stats.zscore(orders["price"])) < 0.3)]
    orders = orders.drop_duplicates(keep='first')

    dataset_id = "EditedReports"
    table_name = "Order"
    insertintobq_keywordResult(dataset_id,table_name,orders)
    return

In [5]:
def paymentsInsert():
    global payments
    file = "file_payments.csv"
    dtypes = {"order_id":'str','payment_types':'Int64','payment_method':'str',"payment_value":'float','currency':'str'}
    parse_dates = []

    payments = pd.read_csv(file, sep=',', dtype=dtypes, parse_dates=parse_dates)

    # Detect and exclude outliers in a pandas DataFrame
    payments = payments[(np.abs(stats.zscore(payments["payment_value"])) < 3)]
    payments = payments.drop_duplicates(keep='first')

    dataset_id = "EditedReports"
    table_name = "Payment"
    insertintobq_keywordResult(dataset_id,table_name,payments)
    return

# Drive Part

In [6]:
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth.transport.requests import Request
from oauth2client.service_account import ServiceAccountCredentials


CLIENT_SECRET_FILE = "key.json"
API_SERVICE_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(CLIENT_SECRET_FILE, SCOPES)
service = build(API_SERVICE_NAME, API_VERSION, credentials=credentials, cache_discovery=False)

In [7]:
def downloadFile(file_id):
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))

In [8]:
page_token = None
while True:
    response = service.files().list(q="'1IAbub9-LFMMauFUo93w1cgKu9rReECEC' in parents and mimeType='text/csv'",
                                            spaces='drive',
                                            fields='nextPageToken, files(id, name,modifiedTime)',
                                            pageToken=page_token).execute()
    for file in response.get('files', []):
        id = file.get('id')
        name = file.get('name')
        modifiedTime = datetime.strptime(file.get('modifiedTime'), "%Y-%m-%dT%H:%M:%S.%f%z")

        now = localtz.localize(datetime.now())

        duration = now - modifiedTime 
        duration_in_s = duration.total_seconds()  
        # Process change
        # print(f'Found file: id: {id} | Name: {name} | ModifiedTime: {modifiedTime}')
        minutes = int(duration_in_s/60)
        if minutes < 1:
            downloadFile(id)
            print(minutes)

            if name == 'file_payments.csv':
                paymentsInsert()
            if name == 'file_orders.csv':
                ordersInsert()
            if name == 'file_customers.csv':
                customersInsert()
    
    page_token = response.get('nextPageToken', None)
    if page_token is None:
        break  


---
# CASE PART 1

What are the top product categories most
commonly purchased by ﬁrst-time customers?

In [9]:
customersInsert()
ordersInsert()
paymentsInsert()

tamamlandı
tamamlandı
tamamlandı


In [ ]:
print(customers.head(1))
print(orders.head(1))
print(payments.head(1))

In [ ]:
total = orders.merge(payments, how='left', on=["order_id"],suffixes=("_orders", "_payments"))
total = total.merge(customers, how='left', on=["customer_id"],suffixes=("_total", "_payments"))

### What are the top product categories most commonly purchased by ﬁrst-time customers?
SQL
```
with a as (
    select 
    o.customer_id, min(o.order_approved_at) as order_approved_at
    from `data-339617.EditedReports.Order` as o
    group by  o.customer_id
)
select o.product_category_name, 
    count(*) as count_,
    --SUM(COUNT(*)) OVER () AS Total_number_of_purchase,
    ROUND((COUNT(*) / SUM(COUNT(*)) OVER ())*100,2) AS Percentage_of_purchase
from  `data-339617.EditedReports.Order` as o
inner join a on o.customer_id = a.customer_id and o.order_approved_at = a.order_approved_at
group by o.product_category_name
order by count_ desc
```

### How large is the segment of customers with an average payment value over 200 SEK?

```
with a as 
-- a means order price table
(select 
    o.order_id,
    o.customer_id,
    avg(CASE 
        --1 EUR ~= 10.5 SEK 
        when p.currency = 'EUR' then p.payment_value*10.5 
        else p.payment_value
    end) as price

from  `data-339617.EditedReports.Order` as o
inner join  `data-339617.EditedReports.Payment` as p on o.order_id = p.order_id
--where o.order_id = '15d972c5dd4501e8d219acbe1bea1519'
group by o.order_id , o.customer_id),

segment as (
    select a.customer_id, 
        avg(price) as averageCustomerOrderValue,
        case when avg(price) <200 then 'Below 200'
        else 'Over 200'
        end as segment
    from a 
    group by a.customer_id
)
select s.segment, 
    count(*) as number_of_customer,
    ROUND((COUNT(*) / SUM(COUNT(*)) OVER ()),2) AS percentage
from segment as s
group by  s.segment
```

### Does the order value increase as the customers place more orders?
https://docs.google.com/spreadsheets/d/1BMm7WdEtahSsBLV9jAKzlNHyMfU4NwqNrXcSaV5IlFI/edit#gid=795405939

```
with a as (select p.order_id, 
    avg(p.payment_value) as avg_cart_value, 
    count(*) as number_of_cart_items,
    min(o.order_approved_at) as date
from  `data-339617.EditedReports.Payment` as p
inner join  `data-339617.EditedReports.Order` as o on o.order_id = p.order_id
group by p.order_id 
)

select number_of_cart_items, round(avg(avg_cart_value),2) as avgValue
from a
group by number_of_cart_items
order by avgValue desc
```

In [14]:
orders.describe()

,order_item_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,152677.000000,1.526770e+05,1.526770e+05,150492.000000,150492.000000,150492.000000,152659.000000,152659.000000,152659.000000,152659.000000
mean,1.200449,3.657156e+06,2.902604e+05,48.797604,792.160102,2.205905,2097.343747,30.090922,16.575243,22.873882
std,0.711564,1.045956e+09,6.180656e+07,9.987869,653.253108,1.728801,3742.475718,16.252474,13.461716,11.656655
min,1.000000,1.700000e+00,0.000000e+00,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,1.000000,9.980000e+01,2.594000e+01,42.000000,348.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,1.000000,2.198000e+02,3.256000e+01,52.000000,608.000000,1.000000,675.000000,25.000000,13.000000,20.000000
75%,1.000000,5.999800e+02,4.230000e+01,57.000000,990.000000,3.000000,1800.000000,38.000000,20.000000,30.000000
max,21.000000,3.920000e+11,1.599000e+10,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


In [46]:
# orders[(np.abs(stats.zscore(orders)) < 3).all(axis=1)]
orders[(np.abs(stats.zscore(orders["price"])) < 0.3)].loc[orders['product_id'] == '241241093bd6986615756b4d8a01de8f']


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_item_id,product_id,seller_id,price,...,currency,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,std
54772,1db6346bb9b84ddccb41b287b6ca1fd8,92b8cf3a3401ec888202af5cbf218819,delivered,2017-09-30 00:56:26+00:00,2017-09-30 01:07:15+00:00,2017-10-02 21:32:19+00:00,1,241241093bd6986615756b4d8a01de8f,43f8c9950d11ecd03a0304a49e010da6,355.98,...,SEK,consoles_games,41,1292,1,200,20,3,18,0.003496
